## Your data processor should be able to ingest a pre-defined data source and perform at least three of these operations:

1. Fetch / download / retrieve a remote data file by URL, or ingest a local file mounted. Suggestions for remote data sources are listed at the end of this document. 

In [1]:
import os
import pymysql
import mysql.connector
from sqlalchemy import create_engine
import csv
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#Source: https://www.kaggle.com/datasets/snehaanbhawal/netflix-tv-shows-and-movie-list
#Read in csv as data frame
netflix_data = pd.read_csv('netflix_list.csv') 
netflix_data.head(2)

,imdb_id,title,popular_rank,certificate,startYear,endYear,episodes,runtime,type,orign_country,language,plot,summary,rating,numVotes,genres,isAdult,cast,image_url
0,tt4052886,Lucifer,1,15,2016.0,NaN,93.0,42,tvSeries,United States,English,Lucifer Morningstar has decided he's had enoug...,"Lucifer Morningstar, bored from his sulking li...",8.1,250884.0,"Crime,Drama,Fantasy",0,"['Tom Ellis', 'Lauren German', 'Lesley-Ann Bra...",https://m.media-amazon.com/images/M/MV5BNzY1Yj...
1,tt0993840,Army of the Dead,2,18,2021.0,NaN,NaN,148,movie,United States,English,"Following a zombie outbreak in Las Vegas, a gr...","With the abandoned, walled city of Las Vegas o...",5.8,110780.0,"Action,Crime,Horror",0,"['Dave Bautista', 'Ella Purnell', 'Ana de la R...",https://m.media-amazon.com/images/M/MV5BNGY0Nz...


2. Convert the general format and data structure of the data source (from JSON to CSV, from CSV to JSON, from JSON into a SQL database table, etc. I want the option to convert any source to any target. So, if I get a CSV as an input, I want the user to choose an output)

In [3]:
#Get input from user, desired file type - Note: file is already in CSV
while True:
    file_type = input("Please select one of the following: JSON, CSV, or SQL.")
    if (file_type == 'JSON'):
       break
    if (file_type == 'CSV'):
       break
    if (file_type == 'SQL'):
       break
    else:
       print('Valid input, please: JSON, CSV, or SQL')

# Source: https://www.geeksforgeeks.org/convert-csv-to-json-using-python/
# Function to convert a CSV to JSON
# Takes the file paths as arguments
def make_json(csvFilePath, jsonFilePath):
     
    # create a dictionary
    data = {}
     
    # Open a csv reader called DictReader
    with open(csvFilePath, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)
         
        # Convert each row into a dictionary
        # and add it to data
        for rows in csvReader:
             
            # Assuming a column named 'No' to
            # be the primary key
            key = rows['title']
            data[key] = rows
 
    # Open a json writer, and use the json.dumps()
    # function to dump data
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
        jsonf.write(json.dumps(data, indent=4))
        
import sqlite3
conn = sqlite3.connect("netflix_list.db")
c = conn.cursor()

#if user wants to use csv -> keep the same
if (file_type == "CSV"):
    netflix_data = netflix_data
    
#if user wants to use json -> call make_json function, then read json file into df and make necessary changes
#JSON file will be created in folder
if (file_type == "JSON"):
    csvFilePath = r'netflix_list.csv'
    jsonFilePath = r'netflix_list.json'
    make_json(csvFilePath, jsonFilePath)
    netflix_data = pd.read_json("netflix_list.json")
    netflix_data = netflix_data.transpose() #switches columns and rows
    netflix_data.reset_index(drop=True, inplace=True)
    nan_value = float("NaN")
    netflix_data.replace("", nan_value, inplace=True) #replace empty string values with NaN

#if user wants to use sql -> use to_sql function, then read in the .db file and make necessary changes
#.db SQL file will be created in folder
if (file_type == "SQL"):
    netflix_data.to_sql("netflix_data", conn, if_exists = 'replace')
    netflix_data = pd.read_sql('SELECT * FROM netflix_data', conn)
    netflix_data.drop('index', inplace=True, axis=1)

Please select one of the following: JSON, CSV, or SQL.SQL


3. Modify the number of columns from the source to the destination, reducing or adding columns.

In [5]:
# Drop unecessary columns
netflix_data.drop('certificate', inplace=True, axis=1)
netflix_data.drop('endYear', inplace=True, axis=1)
netflix_data.drop('runtime', inplace=True, axis=1)
netflix_data.drop('plot', inplace=True, axis=1)
netflix_data.drop('summary', inplace=True, axis=1)
netflix_data.drop('cast', inplace=True, axis=1)
netflix_data.drop('image_url', inplace=True, axis=1)

# Drop rows with empty data
netflix_data.dropna(inplace=True)

# Reset indicies
netflix_data.reset_index(drop=True, inplace=True)

# Truncate rows after 500 - needed to delete rows becasue SQL could not load all the data.
netflix_data = netflix_data[:501]
netflix_data.head(2)

,imdb_id,title,popular_rank,startYear,episodes,type,orign_country,language,rating,numVotes,genres,isAdult
0,tt4052886,Lucifer,1,2016.0,93.0,tvSeries,United States,English,8.1,250884.0,"Crime,Drama,Fantasy",0
1,tt7255502,The Kominsky Method,3,2018.0,22.0,tvSeries,United States,English,8.2,28795.0,"Comedy,Drama",0


4.	The converted (new) file should be written to disk (local file) or written to a SQL database.

In [6]:
# Connect and create database
host_name = "localhost"
host_ip = "127.0.0.1"
port = "3306"
user_id = "root"
pwd = "Quynhanh01"

conn = pymysql.connect(host=host_name, user=user_id, password=pwd)
cursor = conn.cursor()

try:
    cursor.execute("CREATE DATABASE netflix")
    print("Database is created")
except:
    print("Error while connecting to MySQL")
    
conn.close()

Database is created


In [7]:
# Source: https://www.projectpro.io/recipes/connect-mysql-python-and-import-csv-file-into-mysql-and-create-table

# Connect to netflix database and create new table, then iterate through data to insert into new table
# Note: data is truncated becasue the file is too large for mySQL
db_name = "netflix"
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
cursor = conn.cursor()

try:
    cursor.execute("select database();")
    record = cursor.fetchone()
    print("You're connected to database: ", record)
    cursor.execute('DROP TABLE IF EXISTS netflix_data;')
    print('Creating table....')
# in the below line please pass the create table statement which you want #to create
    cursor.execute("CREATE TABLE netflix_data(imdb_id varchar(255),title varchar(255),popular_rank int,startYear int,episodes int,type varchar(255),origin_country varchar(255),language varchar(255),rating int,numVotes int,genres varchar(255),isAdult int)")
    print("Table is created....")
    #loop through the data frame
    for i,row in netflix_data.iterrows():
        #here %S means string values 
        sql = "INSERT INTO netflix.netflix_data VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        cursor.execute(sql, tuple(row))
        print("Record inserted")
        # the connection is not auto committed by default, so we must commit to save our changes
        conn.commit()
except:
        print("Error while connecting to MySQL")
conn.close()

You're connected to database:  ('netflix',)
Creating table....
Table is created....
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Reco

5.	Generate a brief summary of the data file ingestion including: 
<br>
a)	Number of records 
<br>
b)	Number of columns 

In [8]:
numRecords = len(netflix_data)
numColumns = len(netflix_data.columns)

print("Number of records = " + str(numRecords))
print("Number of columns = " + str(numColumns))

Number of records = 501
Number of columns = 12
